In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from pickle import dump
import numpy as np
import h5py
import flows as flows
from layers import MaskedConv2d, MaskedLinear, CNN_Flow_Layer, Dilation_Block

In [2]:
     class ConvNet(nn.Module):
        def __init__(self):
            super(ConvNet, self).__init__()

            self.latent_dim = 20
            self.batch_size = 100                
            self.q_z_output_dim = 128
            self.z_size = 20
            self.test_mode = 0
    
            self.q_z_nn = nn.Sequential(
                  nn.Conv2d(1, 32, kernel_size=(3,4), stride=(1), padding=(0)),
                  nn.BatchNorm2d(32),
                  nn.ReLU(),
                  nn.Conv2d(32, 16, kernel_size=(5,1), stride=(1), padding=(0)),
                  nn.BatchNorm2d(16),
                  nn.ReLU(),
                  nn.Conv2d(16, 8, kernel_size=(7,1), stride=(1), padding=(0)),
                  nn.BatchNorm2d(8),
                  nn.ReLU()
                  ) 
            #
            self.mult_bn1 = nn.BatchNorm1d(161) 
            self.dense1 = nn.Linear(161,self.q_z_output_dim) #161 - full(13+3)+met, 41-4LJ, 216 - full(13+4)+met+mult
            self.dnn_bn1 = nn.BatchNorm1d(self.q_z_output_dim)
            self.q_z_mean = nn.Linear(self.q_z_output_dim, self.latent_dim)
            self.q_z_logvar = nn.Linear(self.q_z_output_dim, self.latent_dim)
            #
            self.dense3 = nn.Linear(self.latent_dim, self.q_z_output_dim)
            self.dnn_bn3 = nn.BatchNorm1d(self.q_z_output_dim)
            self.dense4 = nn.Linear(self.q_z_output_dim, 161)
            self.dnn_bn4 = nn.BatchNorm1d(161)
            #
            self.p_x_nn = nn.Sequential(
                  nn.ConvTranspose2d(64, 64, kernel_size=(7,1), stride=(1), padding=(0)),
                  nn.BatchNorm2d(64),
                  nn.ReLU(),
                  nn.ConvTranspose2d(64, 32, kernel_size=(5,1), stride=(1), padding=(0)),
                  nn.BatchNorm2d(32),
                  nn.ReLU(),
                  nn.ConvTranspose2d(32, 16, kernel_size=(3,4), stride=(1), padding=(0))
                  ) 
            # log-det-jacobian = 0 without flows
            self.ldj = 0
            
            self.encoder = nn.Sequential(
                nn.Linear(5, 50),
                nn.LeakyReLU(True),
                nn.Linear(50, 30),
                nn.LeakyReLU(True),
                nn.Linear(30, 20),
                nn.LeakyReLU(True),
                nn.Linear(20, self.q_z_output_dim)
                )
            
            self.decoder = nn.Sequential(
                nn.Linear(self.q_z_output_dim, 20),
                nn.LeakyReLU(True),
                nn.Linear(20, 30),
                nn.LeakyReLU(True),
                nn.Linear(30, 50),
                nn.LeakyReLU(True),
                nn.Linear(50, 4)
                )
    
        def encode(self, x, y):
          
            out = torch.cat((x, y),axis=1)
            out = self.encoder(out)
            # dense Layer 2
            mean  = self.q_z_mean(out)
            logvar = self.q_z_logvar(out)
            return mean, logvar
    
        def decode(self, z, y):

            out = torch.cat((z, y),axis=1)
            out = self.mult_bn1(out)
            # dense Layer 3
            out = self.dense3(out)
            out = self.dnn_bn3(out)
            out = torch.relu(out)
            # dense Layer 4
            out = self.dense4(out)
            out = self.dnn_bn4(out)
            out = torch.relu(out)
            # DeConv
            out = self.p_x_nn(out)
            out = nn.Sigmoid()
            
            return out
    
        def reparameterize(self, mean, logvar):
            z = mean + torch.randn_like(mean) * torch.exp(0.5 * logvar)
            return z
    
        def forward(self, x, y):
            mean, logvar = self.encode(x, y)
            z = self.reparameterize(mean, logvar)
            out = self.decode(z)
            return out, mean, logvar, self.ldj, z, z

In [3]:
class CNN_Flow(nn.Module):    
    def __init__(self, dim, cnn_layers, kernel_size, test_mode=0, use_revert=True):
        super(CNN_Flow, self).__init__()

        # prepare reversion matrix
        self.usecuda = True
        self.use_revert = use_revert
        self.R = Variable(torch.from_numpy(np.flip(np.eye(dim), axis=1).copy()).float(), requires_grad=False)
        if self.usecuda:
            self.R = self.R.cuda()
        
        self.layers = nn.ModuleList()
        for i in range(cnn_layers):

            block = Dilation_Block(dim, kernel_size, test_mode)
            self.layers.append(block)
        
    def forward(self, x):
        logdetSum = 0
        output = x
        for i in range(len(self.layers)):
            output, logdet = self.layers[i](output)
            # revert the dimension of the output after each block
            if self.use_revert:
                z = output.mm(self.R)
            logdetSum += logdet

        return z, logdetSum

In [4]:
 class ConvFlowVAE(ConvNet):   
    def __init__(self):
        super(ConvFlowVAE, self).__init__()

        # Initialize log-det-jacobian to zero
        self.log_det_j = 0
        self.num_flows = 4#args.num_flows # 6 for chan1
        self.kernel_size = 10

        flow_k = flows.CNN_Flow

        # Normalizing flow layers
        self.flow = flow_k(dim=self.latent_dim, cnn_layers=self.num_flows, kernel_size=self.kernel_size, test_mode=self.test_mode)

    def forward(self, x, y):
        self.met = y

        # mean and variance of z
        z_mu, z_var = self.encode(x, y)
        # sample z
        z_0 = self.reparameterize(z_mu, z_var)

        # Normalizing flows
        z_k, logdet = self.flow(z_0)

        x_decoded = self.decode(z_k)

        return x_decoded, z_mu, z_var, self.log_det_j, z_0, z_k

In [5]:
outerdata_train = np.load("/global/u2/a/agarabag/anomoly_studies/CATHODE/separated_data/outerdata_train.npy")
outerdata_test = np.load("/global/u2/a/agarabag/anomoly_studies/CATHODE/separated_data/outerdata_test.npy")

outerdata_train = outerdata_train[outerdata_train[:,5]==0]
outerdata_test = outerdata_test[outerdata_test[:,5]==0]
print ("TTTTTTTT: " ,  outerdata_test.shape)
nFeat = 4

data_train = outerdata_train[:,1:nFeat+1]
print('shape of data_train: ', data_train.shape)
data_test = outerdata_test[:,1:nFeat+1]
print('shape of data_test: ', data_test.shape)

data = np.concatenate((data_train, data_test), axis=0)
print('shape of data: ', data.shape)

cond_data_train = outerdata_train[:,0]
print('shape of cond_train', cond_data_train.shape)
cond_data_test = outerdata_test[:,0]
print('shape of cond_test', cond_data_test.shape)

cond_data = np.concatenate((cond_data_train, cond_data_test), axis=0)
print('shape of data: ', cond_data.shape)


# In[5]:


max = np.empty(nFeat)
for i in range(0,data.shape[1]):
	max[i] = np.max(np.abs(data[:,i]))
	if np.abs(max[i]) > 0: 
		data[:,i] = data[:,i]/max[i]
	else:
		pass

cond_max = np.max(np.abs(cond_data))
if np.abs(cond_max) > 0:
    cond_data = cond_data/cond_max
else:
    pass


# In[6]:


trainsize = 500000
	
print(np.shape(data))
x_train = data[:trainsize]
x_test = data[trainsize:]
y_train = cond_data[:trainsize]
y_test = cond_data[trainsize:]

# x_train = np.hstack([x_train,y_train.reshape(y_train.shape[0],1)])
# x_test = np.hstack([x_test,y_test.reshape(y_test.shape[0],1)])

image_size = x_train.shape[1]
original_dim = image_size
# print("RRRRRRRR: " , x_trainx.shape)
x_train = np.reshape(x_train, [-1, original_dim])
# print("RRRRRRRR: " , x_train.shape)
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('X', x_train.shape, x_test.shape)
y_train = np.reshape(y_train, [-1, 1])
y_test = np.reshape(y_test, [-1, 1])
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
print('Y', y_train.shape, y_test.shape)


TTTTTTTT:  (378759, 6)
shape of data_train:  (499889, 4)
shape of data_test:  (378759, 4)
shape of data:  (878648, 4)
shape of cond_train (499889,)
shape of cond_test (378759,)
shape of data:  (878648,)
(878648, 4)
X (500000, 4) (378648, 4)
Y (500000, 1) (378648, 1)


In [34]:
batch_size = 100
x_training = torch.utils.data.DataLoader(dataset = x_train, batch_size = batch_size, shuffle=True)
x_testing = torch.utils.data.DataLoader(dataset = x_test, batch_size = batch_size, shuffle=False)

y_training = torch.utils.data.DataLoader(dataset = y_train, batch_size = batch_size, shuffle=True)
y_testing = torch.utils.data.DataLoader(dataset = y_test, batch_size = batch_size, shuffle=False)

In [35]:
# build model
cvae = ConvFlowVAE()
if torch.cuda.is_available():
    cvae.cuda()

In [36]:
cvae

ConvFlowVAE(
  (q_z_nn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 4), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 16, kernel_size=(5, 1), stride=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(16, 8, kernel_size=(7, 1), stride=(1, 1))
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (mult_bn1): BatchNorm1d(161, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=161, out_features=128, bias=True)
  (dnn_bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (q_z_mean): Linear(in_features=128, out_features=20, bias=True)
  (q_z_logvar): Linear(in_features=128, out_features=20, bias=True)
  (dense3): Linear(in_features=20, out_features=128, bias=True)
  (dnn_bn3): Batch

In [ ]:
input_train = x_training.cuda().float()
met_train = y_training.cuda().float()

for epoch in range(self.num_epochs):
    self.x_graph.append(epoch)
    print('Starting to train ...')

    # training
    tr_loss_aux = 0.0
    tr_kl_aux = 0.0
    tr_rec_aux = 0.0
    for y, (x_train, met_tr) in tqdm(enumerate(zip(self.train_loader, self.metTr_loader))):